In [1]:
import mujoco,cv2,pyvista
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import sample_xyzs,rpy2r,r2rpy,r2quat,compute_view_params,get_interp_const_vel_traj, printmd
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline


In [2]:
print ("MuJoCo version:[%s]"%(mujoco.__version__))
xml_path = '../../../asset/visualize_realworld_wo_shelf_pringles.xml'
env = MuJoCoParserClass(name='Place task scene: Office table',rel_xml_path=xml_path,VERBOSE=False, MODE='window')
print(env.MODE)

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('avoiding_object_table').pos = np.array([0.38+0.45,0,0])
env.model.body('base').pos = np.array([-10.18,0,0.79])
env.model.body('right_object_table').pos = np.array([-0.05,-0.80,0])
env.model.body('left_object_table').pos = np.array([-0.05,0.80,0])

# Place objects
tray_pos = np.array([ 0.9, 0.2, 0.75])
tray_gray_pos = np.array([ 0.89, 0., 0.75])
bowl_red_pos = np.array([ 0.82, -0.175, 0.75])
env.model.joint(env.model.body('tray').jntadr[0]).qpos0[:3] = tray_pos
env.model.joint(env.model.body('tray').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([90, 0, 90])))
env.model.joint(env.model.body('tray_gray').jntadr[0]).qpos0[:3] = tray_gray_pos
env.model.joint(env.model.body('tray_gray').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([90, 0, 90])))
env.model.joint(env.model.body('kitchen-bowl-blue').jntadr[0]).qpos0[:3] = bowl_red_pos

env.model.joint(env.model.body('ycb-apple-2').jntadr[0]).qpos0[:3] = bowl_red_pos + np.array([0,0,0.03])
env.model.joint(env.model.body('mug_cup').jntadr[0]).qpos0[:3] = tray_pos + np.array([0.07,-0.01,0.0])
env.model.joint(env.model.body('mug_cup').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([0, 0, 90])))
env.model.joint(env.model.body('hershey_cookie').jntadr[0]).qpos0[:3] = tray_gray_pos + np.array([0.12,0,0.05])
env.model.joint(env.model.body('hershey_cookie').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([90, -90, 0])))

env.model.joint(env.model.body('kitchen-bowl-red').jntadr[0]).qpos0[:3] = np.array([-5,0,0.5])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[:3] = np.array([-9,0,0]) # sorted_pose[2] + spawn_z_offset
env.model.joint(env.model.body('ycb-lemon-2').jntadr[0]).qpos0[:3] = np.array([-5,0,0.5])
env.model.joint(env.model.body('ycb-orange-2').jntadr[0]).qpos0[:3] = np.array([-5,0,0.5])

# Target objects
env.model.joint(env.model.body('pringles').jntadr[0]).qpos0[:3] = np.array([0.1, -0.6, 0.8])

MuJoCo version:[2.3.7]
window


In [3]:
import time

# Init viewer
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True, MODE='window')
env.update_viewer(azimuth=0,distance=0.7,elevation=-30,lookat=[0.7,0.0,1.01],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=False,jointlength=0.25,jointwidth=0.05,jointrgba=[0.2,0.6,0.8,0.6])

# Reset
env.reset()
init_ur_q = np.array([-2.28, -1.03, -2.02,  2.43, -0.81,  0.46])
env.forward(q=init_ur_q, joint_idxs=env.idxs_forward)

for i in range(6):
    if i == 1:      # Apple -> Left Tray
        env.model.joint(env.model.body('ycb-apple-2').jntadr[0]).qpos0[:3] = np.array([0.80, 0.25, 0.75])
    elif i == 2:    # Mug cup -> Center Tray
        env.model.joint(env.model.body('mug_cup').jntadr[0]).qpos0[:3] = np.array([0.90, -0.05, 0.75])
    elif i == 3:    # Hershey -> Left Tray
        env.model.joint(env.model.body('hershey_cookie').jntadr[0]).qpos0[:3] = np.array([1.025, 0.21, 0.80])
    elif i == 4:    # Apple -> Right Tray
        env.model.joint(env.model.body('ycb-apple-2').jntadr[0]).qpos0[:3] = bowl_red_pos + np.array([0,0,0.03])
    elif i == 5:    # Hershey -> Left Tray
        env.model.joint(env.model.body('hershey_cookie').jntadr[0]).qpos0[:3] = np.array([0.85, 0.205, 0.80])

    env.reset()
    init_ur_q = np.array([-2.28, -1.03, -2.02,  2.43, -0.81,  0.46])
    env.forward(q=init_ur_q, joint_idxs=env.idxs_forward)
    start_t = time.time()
    while time.time() - start_t < 3:
        env.step(ctrl=init_ur_q,ctrl_idxs=env.idxs_step)
        # env.step(ctrl=init_ur_q,ctrl_idxs=env.idxs_step)
        # env.model.geom_size[env.model.body('obj_box_03').geomadr[0]] += np.array([0.0001,0.0001,0.0001])
        if not env.is_viewer_alive(): break

        # Render
        if env.loop_every(HZ=200):
            # Compute some poses
            p_tcp,R_tcp = env.get_pR_body(body_name='tcp_link')
            p_cam,R_cam = env.get_pR_body(body_name='camera_center')
            p_base,R_base = env.get_pR_body(body_name='base')
            # Get PCD from a specific view
            p_ego  = p_cam
            p_trgt = p_cam + R_cam[:,2] # + np.array([0,0,-0.2])
            rgb_img,depth_img,pcd,xyz_img = env.get_egocentric_rgb_depth_pcd(
                p_ego=p_ego,p_trgt=p_trgt,rsz_rate=40,fovy=45,BACKUP_AND_RESTORE_VIEW=True)
            env.render(render_every=1)

# Close viewer
env.close_viewer()

Pressed ESC
Quitting.
